# Deep Learning - Project 1
Wojciech Kutak

---

### Cross Attention Network for few-shot learning problem
#### 1. Code

In [1]:
import os
import torch
from torch import nn
import torch.nn.functional as F
from torch.optim import SGD
import torchvision
from torchinfo import summary
from tqdm import tqdm


DATA_PATH = os.path.join("..", "data")
FEW_SHOT_PATH = os.path.join("..", "data_few_shot")

In [2]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride = 1, downsampler = None):
        super().__init__()
        self.downsampler = downsampler

        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.relu = nn.ReLU(inplace=True)
        self.out_channels = out_channels


    def forward(self, x):
        identity = x
        if self.downsampler is not None:
            identity = self.downsampler(identity)

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)

        out = out + identity
        out = self.relu(out)
        return out


class ResNet_32x32(nn.Module):
    def __init__(self, block: nn.Module = ResidualBlock):
        super().__init__()
        self.conv = nn.Conv2d(3, 64, (3, 3), stride=1, padding=1)
        self.bnorm = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)

        self.max_pool1 = nn.MaxPool2d(kernel_size=3, stride=2)
        self.res_layer1 = self._residual_layer(block, 64, 128, 2)

        self.max_pool2 = nn.MaxPool2d(kernel_size=3, stride=2)
        self.res_layer2 = self._residual_layer(block, 128, 256, 2)

        self.max_pool3 = nn.MaxPool2d(kernel_size=3, stride=2)
        self.res_layer3 = self._residual_layer(block, 256, 512, 2)

        self.output_shape = (512, 3, 3)


    def _residual_layer(self, block, in_channels, out_channels, blocks_num, stride=1):
        """Creates a residual layer consisting out of residual blocks"""
        downsampler = None
        if stride != 1 or in_channels != out_channels:
            downsampler = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

        layers = []
        layers.append(block(in_channels, out_channels, stride, downsampler))
        for _ in range(blocks_num - 1):
            layers.append(block(out_channels, out_channels, stride))

        return nn.Sequential(*layers)


    def forward(self, x):
        x = self.relu(self.bnorm(self.conv(x)))

        x = self.res_layer1(self.max_pool1(x))
        x = self.res_layer2(self.max_pool2(x))
        x = self.res_layer3(self.max_pool3(x))

        return x

In [150]:
summary(ResNet_32x32(), (30, 3, 32, 32))

Layer (type:depth-idx)                   Output Shape              Param #
ResNet_32x32                             [30, 512, 3, 3]           --
├─Conv2d: 1-1                            [30, 64, 32, 32]          1,792
├─BatchNorm2d: 1-2                       [30, 64, 32, 32]          128
├─ReLU: 1-3                              [30, 64, 32, 32]          --
├─MaxPool2d: 1-4                         [30, 64, 15, 15]          --
├─Sequential: 1-5                        [30, 128, 15, 15]         --
│    └─ResidualBlock: 2-1                [30, 128, 15, 15]         --
│    │    └─Sequential: 3-1              [30, 128, 15, 15]         8,448
│    │    └─Conv2d: 3-2                  [30, 128, 15, 15]         73,856
│    │    └─BatchNorm2d: 3-3             [30, 128, 15, 15]         256
│    │    └─ReLU: 3-4                    [30, 128, 15, 15]         --
│    │    └─Conv2d: 3-5                  [30, 128, 15, 15]         147,584
│    │    └─BatchNorm2d: 3-6             [30, 128, 15, 15]         2

In [240]:
import math

class FusionLayer(nn.Module):
    def __init__(self, m: int, bottleneck_size: int):
        super(FusionLayer, self).__init__()

        self.temperature = 1.0
        self.m = m
        self.bottleneck_size = bottleneck_size
        self.conv1 = nn.Conv2d(self.m, self.bottleneck_size, kernel_size=1)
        self.bn = nn.BatchNorm2d(self.bottleneck_size)
        self.conv2 = nn.Conv2d(self.bottleneck_size, self.m, kernel_size=1)
        self.relu = nn.ReLU()

        # TODO:
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))


    def forward(self, R: torch.Tensor):
        # print("\nFusionLayer forward")
        # print("R:", R.shape)

        # If R = R_p
        # Dim=3 corresponds to all queries and dim=4 to all support examples
        # We are averaging over all queries to extract info about the mean relevance of
        # certain pixel from support to all other queries
        # Otherwise it's the other way around
        w = torch.mean(R, dim=3)
        # print("w", w.shape)
        w = w.transpose(1, 3)
        # print("w", w.shape)

        w = self.conv1(w)
        # print("w", w.shape)
        w = self.bn(w)
        w = self.relu(w)
        w = self.conv2(w)
        # print("w", w.shape)
        w = w.transpose(1, 3).unsqueeze(3)
        # print("w", w.shape)

        # I guess... Unfortunately not well described in paper
        # I suppose it's a weighted average over query features
        A = torch.mean(w * R, dim=-1)
        A = F.softmax(A / self.temperature, dim=-1) + 1

        # print("A:", A.shape)
        return A

        # print("R shape:", R.shape)

        # b, M, m1, m2 = R.shape
        # assert m1 == m2

        # # Global average pooling
        # w: torch.Tensor = R.mean(dim=-2)
        # print("spatial w:", w.shape)
        # w = w.transpose(1, 3)
        # print("spatial w after:", w.shape)

        # # w = w.unsqueeze(-2)
        # # Meta learner
        # w = self.conv1(w)
        # w = self.bn(w)
        # print("conv1 w:", w.shape)
        # w = self.relu(w)
        # print("relu w:",w.shape)
        # w = self.conv2(w)
        # print("conv2 w:",w.shape)

        # w = w.transpose(1, 3)
        # print("w tra", w.shape)
        # w = w.squeeze((-2, -1))
        # print("w squeeze", w.shape)

        # # Convolution operation
        # # w is now a vector of average class
        # A = self.attention(w, R)
        # print("Attention:", A.shape)


        # return A


    # def attention(self, weights: torch.Tensor, R: torch.Tensor) -> torch.Tensor:

    #     weights_t = weights.transpose(-3, -1).squeeze(1)

    #     b, M, m, h, w = R.shape
    #     R = R.view(b * M, m, h*w)
    #     # print("weights:", weights.shape)
    #     # print("weights_t:", weights_t.shape)
    #     # print("R:", R.shape)

    #     R_mean = torch.bmm(weights_t, R) / self.temperature
    #     # print("R_mean:", R_mean.shape)
    #     R_mean = R_mean.view(b, M, 1, m)
    #     A = F.softmax(R_mean, dim=-1)
    #     return A

In [133]:
width, height = 5, 5
m = width*height
M = 5
batch_size = 3
summary(FusionLayer(m=m, bottleneck_size=15), (batch_size, M, m, height, width))

FusionLayer forward
R shape: torch.Size([3, 5, 25, 5, 5])
spatial w: torch.Size([3, 5, 25, 1, 1])
spatial w after: torch.Size([15, 25, 1, 1])
conv1 w: torch.Size([15, 15, 1, 1])
relu w: torch.Size([15, 15, 1, 1])
conv2 w: torch.Size([15, 25, 1, 1])
weights: torch.Size([15, 25, 1, 1])
weights_t: torch.Size([15, 1, 25])
R: torch.Size([15, 25, 25])
R_mean: torch.Size([15, 1, 25])
Attention: torch.Size([3, 5, 1, 25])


Layer (type:depth-idx)                   Output Shape              Param #
FusionLayer                              [3, 5, 1, 25]             --
├─AdaptiveAvgPool2d: 1-1                 [3, 5, 25, 1, 1]          --
├─Conv2d: 1-2                            [15, 15, 1, 1]            390
├─ReLU: 1-3                              [15, 15, 1, 1]            --
├─Conv2d: 1-4                            [15, 25, 1, 1]            400
Total params: 790
Trainable params: 790
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.01
Input size (MB): 0.04
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.05

In [59]:
a = torch.tensor([1.0, 2.0]).view(1, 2)
b = torch.tensor([[2.0, 3.0, 4.0], [10.0, 11.0, 112.0]]).view(1, 2, 3, 1)
a*b

tensor([[[[  2.,   4.],
          [  3.,   6.],
          [  4.,   8.]],

         [[ 10.,  20.],
          [ 11.,  22.],
          [112., 224.]]]])

In [239]:
class CrossAttentionModule(nn.Module):
    def __init__(self, input_shape: tuple[int,int,int]):
        super(CrossAttentionModule, self).__init__()

        # print("input shape", *input_shape)
        c, h, w = input_shape
        self.fusion_layer = FusionLayer(m=h*w, bottleneck_size=int(h*w*2/3))


    def forward(self, P: torch.Tensor, Q: torch.Tensor):
        # print("\nCross Attention forward")
        b, n_p, c, h, w = P.shape
        n_q = Q.shape[1]
        # print("P", P.shape)
        # print("Q", Q.shape)

        P = P.view(b, n_p, c, h*w)
        Q = Q.view(b, n_q, c, h*w)
        # print("P", P.shape)
        # print("Q", Q.shape)

        P_norm = F.normalize(P, p=2, dim=2)
        Q_norm = F.normalize(Q, p=2, dim=2)
        # print("P_norm", P_norm.shape)
        # print("Q_norm", Q_norm.shape)

        P_norm = P.transpose(2, 3).view(b, n_p, 1, h*w, c)
        Q_norm = Q.view(b, 1, n_q, c, h*w)
        # print("P_norm", P_norm.shape)
        # print("Q_norm", Q_norm.shape)

        # Now it will be broadcasted so all pairs of per batch will be miltiplied
        R_p = torch.matmul(P_norm, Q_norm)
        R_q = R_p.transpose(3, 4)
        # print("R_p", R_p.shape)
        # print("R_q", R_q.shape)

        A_p: torch.Tensor = self.fusion_layer(R_p)
        A_q: torch.Tensor = self.fusion_layer(R_q)

        # print("P", P.shape, "A_p", A_p.shape)
        P = P.unsqueeze(2)
        A_p = A_p.unsqueeze(3)
        # print("P", P.shape, "A_p", A_p.shape)
        P = P * A_p
        # print("P", P.shape)
        P = P.view(b, n_p, n_q, c, h, w)
        # print("P", P.shape)

        # print("Q", Q.shape, "A_q", A_q.shape)
        Q = Q.unsqueeze(1)
        A_q = A_q.unsqueeze(3)
        Q = Q * A_q
        Q = Q.view(b, n_p, n_q, c, h, w)
        # print("Q", Q.shape)

        return P.transpose(1, 2), Q.transpose(1, 2)

    # def forward(self, S: torch.Tensor):
        # P, Q = S[0], S[1, :, 0, :, :, :]
        # print("CrossAttentionModule forward")
        # print(P.shape, Q.shape)
        # assert P.shape == Q.shape
        # b, M, c, h, w = P.shape
        # assert (b, c, h, w) == Q.shape
        # m = h*w
        # # Change representation from tensor c*h*w to c*m (2 dims)
        # P = P.view(b, M, c, m)
        # Q = Q.view(b, 1, c, m)

        # P_norm = F.normalize(P, p=2, dim=2)
        # Q_norm = F.normalize(Q, p=2, dim=2)
        # print("P norm:", P_norm.shape)
        # P_norm_t = P_norm.transpose(-2, -1).unsqueeze(2)
        # Q_norm = Q_norm.unsqueeze(1)
        # print("P norm t:", P_norm_t.shape)
        # print("Q norm:", Q_norm.shape)
        # R_q = torch.matmul(P_norm_t, Q_norm)
        # print("R_q:", R_q.shape)
        # R_p = R_q.transpose(-2, -1)
        # R_q = R_q
        # print("R_q", R_q.shape)
        # print("R_p", R_p.shape)



        # A_p: torch.Tensor = self.fusion_layer(R_p)
        # print()
        # A_q: torch.Tensor = self.fusion_layer(R_q)
        # print()

        # print("P:", P.shape, "A_p:", A_p.shape)
        # P_feat = torch.mul(P, A_p) + P
        # print("P_feat:", P_feat.shape)

        # print("Q:", Q.shape, "A_q:", A_q.shape)
        # Q = Q.expand(b, M, c, m)
        # print("Q exp:", Q.shape, "A_q:", A_q.shape)
        # Q_feat = torch.mul(Q, A_q) + Q
        # print("Q_feat:", Q_feat.shape)
        # return P_feat, Q_feat

In [71]:
b = 1
M = 6
K = 5
c = 512
height, width = 5, 5
can = CrossAttentionModule((c, height, width))

support = torch.ones(M, K, c, height, width)
query = torch.ones(M, c, height, width)
can(support, query)

P norm: torch.Size([6, 5, 512, 25])
P norm t: torch.Size([6, 5, 1, 25, 512])
Q norm: torch.Size([6, 1, 1, 512, 25])
R_q: torch.Size([6, 5, 1, 25, 25])
R_q torch.Size([6, 5, 1, 25, 25])
R_p torch.Size([6, 5, 1, 25, 25])


ValueError: too many values to unpack (expected 4)

In [ ]:
b = 3
m = 5



summary(CrossAttentionModule((512, 5, 5)), (2, b, m, 512, 5, 5))

CrossAttentionModule forward
torch.Size([3, 5, 512, 5, 5]) torch.Size([3, 512, 5, 5])
P norm: torch.Size([3, 5, 512, 25])
P norm t: torch.Size([3, 5, 25, 512])
Q norm: torch.Size([3, 1, 512, 25])
R_q torch.Size([3, 5, 25, 5, 5])
R_p torch.Size([3, 5, 25, 5, 5])
FusionLayer forward
R shape: torch.Size([3, 5, 25, 5, 5])
spatial w: torch.Size([3, 5, 25, 1, 1])
spatial w after: torch.Size([15, 25, 1, 1])
conv1 w: torch.Size([15, 15, 1, 1])
relu w: torch.Size([15, 15, 1, 1])
conv2 w: torch.Size([15, 25, 1, 1])
weights: torch.Size([15, 25, 1, 1])
weights_t: torch.Size([15, 1, 25])
R: torch.Size([15, 25, 25])
R_mean: torch.Size([15, 1, 25])
Attention: torch.Size([3, 5, 1, 25])

FusionLayer forward
R shape: torch.Size([3, 5, 25, 5, 5])
spatial w: torch.Size([3, 5, 25, 1, 1])
spatial w after: torch.Size([15, 25, 1, 1])
conv1 w: torch.Size([15, 15, 1, 1])
relu w: torch.Size([15, 15, 1, 1])
conv2 w: torch.Size([15, 25, 1, 1])
weights: torch.Size([15, 25, 1, 1])
weights_t: torch.Size([15, 1, 25])


Layer (type:depth-idx)                   Output Shape              Param #
CrossAttentionModule                     [3, 5, 512, 25]           --
├─FusionLayer: 1-1                       [3, 5, 1, 25]             --
│    └─AdaptiveAvgPool2d: 2-1            [3, 5, 25, 1, 1]          --
│    └─Conv2d: 2-2                       [15, 15, 1, 1]            390
│    └─ReLU: 2-3                         [15, 15, 1, 1]            --
│    └─Conv2d: 2-4                       [15, 25, 1, 1]            400
├─FusionLayer: 1-2                       [3, 5, 1, 25]             (recursive)
│    └─AdaptiveAvgPool2d: 2-5            [3, 5, 25, 1, 1]          --
│    └─Conv2d: 2-6                       [15, 15, 1, 1]            (recursive)
│    └─ReLU: 2-7                         [15, 15, 1, 1]            --
│    └─Conv2d: 2-8                       [15, 25, 1, 1]            (recursive)
Total params: 790
Trainable params: 790
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.02
Input size (MB): 1.54


In [36]:
A = torch.tensor([[1.0, 2.0], [3.0, 4.0]])
B = torch.tensor([[1.0, 2.0], [3.0, 4.0]])
torch.mul(A, B), torch.matmul(A, B)

(tensor([[ 1.,  4.],
         [ 9., 16.]]),
 tensor([[ 7., 10.],
         [15., 22.]]))

In [108]:
A = torch.tensor([[1.0, 2.0], [3., 4.]])
B = torch.tensor([[7., 11.]])
A*B

tensor([[ 7., 22.],
        [21., 44.]])

In [256]:
class CrossAttentionNetwork(nn.Module):
    def __init__(self, cam: CrossAttentionModule = None, n_classes: int = 6):
        super(CrossAttentionNetwork, self).__init__()

        self.embedding = ResNet_32x32()
        self.cam = cam if cam is not None else CrossAttentionModule(self.embedding.output_shape)
        self.classifer = nn.Conv2d(self.embedding.output_shape[0], n_classes, kernel_size=1)
        self.n_classes = n_classes


    def forward(self,
                support: torch.Tensor, query: torch.Tensor,
                y_support: torch.Tensor, y_query: torch.Tensor
    ):
        # n_support = number of classes * number of examples per class
        b, n_support, c, h, w = support.shape

        n_y_support_classes = y_support.shape[-1]

        # n_query = number of queries to classify
        n_query = query.size(1)

        assert support.shape[0] == query.shape[0] and \
            support.shape[2:] == query.shape[2:]

        # Embed all support and query images at once
        support = support.view(-1, c, h, w)
        query = query.view(-1, c, h, w)
        X = torch.cat((support, query), dim=0)
        # E is of shape (b*(n_support + n_query), c, h, w)
        E: torch.Tensor = self.embedding(X)
        # print("E", E.shape)

        E_support = E[:b*n_support]
        # Flattening images from (c,h,w) to (c*h*w)
        E_support = E_support.view(b, n_support, -1)
        # print("E_support", E_support.shape)
        y_support = y_support.transpose(1, 2)
        # print("y_supp", y_support.shape)

        # Smart way of averaging embedding vectors, generalized for unbalanced number of
        # support images wrt to class
        E_support_mean = torch.bmm(y_support, E_support)
        # print("y sum", torch.sum(y_support, dim=2, keepdim=True).shape)
        y_support_class_count = torch.sum(y_support, dim=2, keepdim=True).expand_as(E_support_mean)
        # print("y_support_class_count", y_support_class_count.shape)
        E_support_mean = torch.div(E_support_mean, y_support_class_count)
        # print("E_support mean", E_support_mean.shape)
        # print(*E.shape[1:])
        E_support_mean = E_support_mean.view(b, n_y_support_classes, *E.shape[1:])

        E_query = E[b*n_support:]
        E_query = E_query.view(b, n_query, *E.shape[1:])

        # Embedding with saturated features which are common between
        # each support class and query sample
        # E_support_mean/E_query: (b, n_query, n_support, c, h, w)
        E_support_mean, E_query = self.cam(E_support_mean, E_query)
        # print("\nE_support_mean:", E_support_mean.shape)
        # print("E_query:", E_query.shape)

        # Computing mean activations across height and width of image (means of active maps)
        E_support_mean = torch.mean(E_support_mean, dim=(-2, -1))
        # E_query = torch.mean(E_query, dim=(-2, -1))
        # print("E_support_mean:", E_support_mean.shape)
        # print("E_query:", E_query.shape)

        if self.training:
            return self._classification(E_support_mean, E_query, y_query)
        else:
            return self._test(E_support_mean, E_query)


    def _classification(self, P: torch.Tensor, Q: torch.Tensor, y_query: torch.Tensor):
        # print("\n_classification")
        # Nearest neighbor classification
        # Compute cosine distance
        P_norm = F.normalize(P, p=2, dim=3)
        Q_norm  = F.normalize(Q, p=2, dim=3)
        # print("P_norm", P_norm.shape)
        # print("Q", Q.shape)

        P_norm = P_norm.unsqueeze(-1)
        P_norm = P_norm.unsqueeze(-1)
        # print("P_norm", P_norm.shape)
        # print("Q_norm", Q_norm.shape)


        similarity_score = torch.sum(P_norm * Q_norm, dim=3)
        # similarity_score = similarity_score.view(-1, *similarity_score.shape[2:])
        # print("similarity_score", similarity_score.shape)

        # Global classification
        b, n_query, n_support, c, h, w = Q.shape
        # print("\nQ", Q.size())
        Q = Q.reshape(b, n_query, self.n_classes, -1)
        Q = Q.transpose(2, 3)
        # print("Q", Q.size())
        y_query = y_query.unsqueeze(-1)
        # print("y_query:", y_query.shape)
        y_pred = torch.matmul(Q, y_query)
        # print("y_pred:", y_pred.shape)
        y_pred = y_pred.view(b*n_query, c, h, w)
        # print("y_pred:", y_pred.shape)
        y_pred = self.classifer(y_pred)
        # print("y_pred:", y_pred.shape)
        y_pred = y_pred.view(b, n_query, self.n_classes, h, w)

        return y_pred, similarity_score


    def _test(self, P: torch.Tensor, Q: torch.Tensor):
        # Global average pooling over spatial dimensions
        Q = Q.mean(-1)
        Q = Q.mean(-1)

        P_norm = F.normalize(P, p=2, dim=-1)
        Q_norm = F.normalize(Q, p=2, dim=-1)
        similarity_scores = torch.sum(P_norm*Q_norm, dim=-1)
        return similarity_scores


In [257]:
b = 1
n_way, n_shot = 5, 5
c, height, width = 3, 32, 32

n_query = 15

support = torch.ones((b, n_way*n_shot, c, height, width))
y_support = torch.randint(0, n_way, (b, n_way*n_shot))
y_support_ohe = F.one_hot(y_support, n_way).type(torch.float)

query = torch.randn((b, n_query, c, height, width))
y_query = torch.randint(0, n_way, (b, n_query))
y_query_ohe = F.one_hot(y_query, n_way).type(torch.float)


can2test = CrossAttentionNetwork(n_classes=n_way)
y_pred2, sim_score2 = can2test(support, query, y_support_ohe, y_query_ohe)
y_pred2.shape, sim_score2.shape

(torch.Size([1, 15, 5, 3, 3]), torch.Size([1, 15, 5, 3, 3]))

In [223]:
b = 1
n_way, n_shot = 5, 5
c, height, width = 3, 32, 32

n_query = 15

support = torch.ones((b, n_way*n_shot, c, height, width))
y_support = torch.randint(0, n_way, (b, n_way*n_shot))
y_support_ohe = F.one_hot(y_support, n_way).type(torch.float)

query = torch.randn((b, n_query, c, height, width))
y_query = torch.randint(0, n_way, (b, n_query))
y_query_ohe = F.one_hot(y_query, n_way).type(torch.float)
# y_support = torch.randint(0, n_way, (b, n_query))
# y_support_ohe = F.one_hot(y_support, n_way)


can2test = CrossAttentionNetwork(n_classes=n_way)
can2test(support, query, y_support_ohe, y_query_ohe)

E torch.Size([40, 512, 3, 3])
E_support torch.Size([1, 25, 4608])
y_supp torch.Size([1, 5, 25])
y sum torch.Size([1, 5, 1])
y_support_class_count torch.Size([1, 5, 4608])
E_support mean torch.Size([1, 5, 4608])
512 3 3

Cross Attention forward
P torch.Size([1, 5, 512, 3, 3])
Q torch.Size([1, 15, 512, 3, 3])
P torch.Size([1, 5, 512, 9])
Q torch.Size([1, 15, 512, 9])
P_norm torch.Size([1, 5, 512, 9])
Q_norm torch.Size([1, 15, 512, 9])
P_norm torch.Size([1, 5, 1, 9, 512])
Q_norm torch.Size([1, 1, 15, 512, 9])
R_p torch.Size([1, 5, 15, 9, 9])
R_q torch.Size([1, 5, 15, 9, 9])

FusionLayer forward
R: torch.Size([1, 5, 15, 9, 9])
w torch.Size([1, 5, 15, 9])
w torch.Size([1, 9, 15, 5])
w torch.Size([1, 6, 15, 5])
w torch.Size([1, 9, 15, 5])
w torch.Size([1, 5, 15, 1, 9])
A: torch.Size([1, 5, 15, 9])

FusionLayer forward
R: torch.Size([1, 5, 15, 9, 9])
w torch.Size([1, 5, 15, 9])
w torch.Size([1, 9, 15, 5])
w torch.Size([1, 6, 15, 5])
w torch.Size([1, 9, 15, 5])
w torch.Size([1, 5, 15, 1, 9])
A

(tensor([[[[-4.5964e-01, -6.7759e-01, -1.9571e+00],
           [-1.5642e+00, -1.6503e+00, -2.4744e+00],
           [-4.6587e+00, -3.3465e+00, -1.6853e+00]],
 
          [[-1.8017e+00, -1.5339e+00, -4.1703e-02],
           [-1.7650e+00, -2.0061e+00, -1.2024e+00],
           [-3.2324e+00, -2.1887e+00, -9.3838e-01]],
 
          [[ 3.9101e-01,  5.7364e-01, -2.7385e-02],
           [-9.0061e-01, -1.0135e+00,  7.5178e-01],
           [-5.4327e-01, -1.7640e-01, -6.1134e-01]],
 
          [[ 9.8394e-01,  7.5506e-01,  9.8741e-01],
           [ 6.5876e-01,  4.0617e-01, -1.1633e+00],
           [-1.6457e+00,  5.9972e-02,  1.9820e-01]],
 
          [[-4.3297e-01,  1.4053e+00,  5.0144e-01],
           [-3.4192e-01, -3.5208e-02,  1.0980e+00],
           [-7.9379e-01, -1.0404e+00,  8.9025e-01]]],
 
 
         [[[-7.4148e-01, -1.0890e+00, -2.5846e-01],
           [-4.8078e-01, -8.7759e-01, -2.4776e+00],
           [-7.5128e-01, -6.8779e-01, -1.2025e+00]],
 
          [[-2.8457e+00, -1.1334e+00, -8.65

In [84]:
a = torch.ones(2, 1, 2, 2)
b = torch.ones(1, 2, 2, 2)
a[0] = a[0] * 2
a[1] = a[1] * 3
b[:,0] = b[:,0] * 5
b[:,1] = b[:,1] * 7
print(a,b)
print(torch.matmul(a,b).shape)
torch.matmul(a,b)

tensor([[[[2., 2.],
          [2., 2.]]],


        [[[3., 3.],
          [3., 3.]]]]) tensor([[[[5., 5.],
          [5., 5.]],

         [[7., 7.],
          [7., 7.]]]])
torch.Size([2, 2, 2, 2])


tensor([[[[20., 20.],
          [20., 20.]],

         [[28., 28.],
          [28., 28.]]],


        [[[30., 30.],
          [30., 30.]],

         [[42., 42.],
          [42., 42.]]]])

In [163]:
summary(CrossAttentionNetwork(), (2, 32, 5, 5, 3, 32, 32))

input shape 512 3 3
Support:  torch.Size([32, 5, 5, 3, 32, 32])
query:  torch.Size([32, 3, 32, 32])
CrossAttentionModule forward
torch.Size([32, 5, 512, 3, 3]) torch.Size([32, 512, 3, 3])
P norm: torch.Size([32, 5, 512, 9])
P norm t: torch.Size([32, 5, 9, 512])
Q norm: torch.Size([32, 1, 512, 9])
R_q torch.Size([32, 5, 9, 3, 3])
R_p torch.Size([32, 5, 9, 3, 3])
FusionLayer forward
R shape: torch.Size([32, 5, 9, 3, 3])
spatial w: torch.Size([32, 5, 9, 1, 1])
spatial w after: torch.Size([160, 9, 1, 1])
conv1 w: torch.Size([160, 6, 1, 1])
relu w: torch.Size([160, 6, 1, 1])
conv2 w: torch.Size([160, 9, 1, 1])
weights: torch.Size([160, 9, 1, 1])
weights_t: torch.Size([160, 1, 9])
R: torch.Size([160, 9, 9])
R_mean: torch.Size([160, 1, 9])
Attention: torch.Size([32, 5, 1, 9])

FusionLayer forward
R shape: torch.Size([32, 5, 9, 3, 3])
spatial w: torch.Size([32, 5, 9, 1, 1])
spatial w after: torch.Size([160, 9, 1, 1])
conv1 w: torch.Size([160, 6, 1, 1])
relu w: torch.Size([160, 6, 1, 1])
conv2 

Layer (type:depth-idx)                        Output Shape              Param #
CrossAttentionNetwork                         [32, 5, 512, 9]           --
├─ResNet_32x32: 1-1                           [800, 512, 3, 3]          --
│    └─Conv2d: 2-1                            [800, 64, 32, 32]         1,792
│    └─BatchNorm2d: 2-2                       [800, 64, 32, 32]         128
│    └─ReLU: 2-3                              [800, 64, 32, 32]         --
│    └─MaxPool2d: 2-4                         [800, 64, 15, 15]         --
│    └─Sequential: 2-5                        [800, 128, 15, 15]        --
│    │    └─ResidualBlock: 3-1                [800, 128, 15, 15]        230,400
│    │    └─ResidualBlock: 3-2                [800, 128, 15, 15]        295,680
│    └─MaxPool2d: 2-6                         [800, 128, 7, 7]          --
│    └─Sequential: 2-7                        [800, 256, 7, 7]          --
│    │    └─ResidualBlock: 3-3                [800, 256, 7, 7]          919,552
│

In [31]:
class CANLoss(nn.Module):
    def __init__(self, can: CrossAttentionNetwork, lamb: float = 0.5, n_classes: int = 5):
        super(CANLoss, self).__init__()

        self.can = can
        self.lamb = lamb
        self.classifier = nn.LazyConv1d(out_channels=n_classes, kernel_size=1)


    def cosine_dist(self, P_gap: torch.Tensor, Q_features: torch.Tensor) -> torch.Tensor:
        # print("\ncosine_dist")
        # print("Q_features",  Q_features.shape)
        # Q_features: (b, c, m)
        b, c, m = Q_features.shape
        # print("P_gap",  P_gap.shape)
        # P_gap: (b, c)
        P_gap_exp = P_gap.expand(m, b, c).transpose(0, 1).transpose(1, 2)
        # print("P_gap_exp", P_gap_exp.shape)
        cos_dist = F.cosine_similarity(P_gap_exp, Q_features, dim=-2)
        # print("cos_dist", cos_dist.shape)
        return cos_dist


    def L1_loss(self, P_sig: torch.Tensor, Q_sig: torch.Tensor, y_true: torch.Tensor):
        # P_features: (b, c, m)
        # Q_features: (b, c, m)
        # print("l1 loss")
        # print("P_features:", P_sig.shape)
        # print("Q_features:", Q_sig.shape)


        P_gap = torch.mean(P_sig, dim=-1)
        # P_gap: (b, c)
        distances = self.cosine_dist(P_gap, Q_sig)
        # distances: (b, m)
        # print("Distances:", distances.shape)

        # print("distances:", distances)
        likelihoods = -torch.log(F.softmax(-distances, dim=-1))
        # print("likelihoods", likelihoods)
        # print("likelihoods", likelihoods.shape)
        l1 = torch.sum(likelihoods)
        # print("l1", l1)
        return l1


    def L2_loss(self, Q_sig: torch.Tensor, y_true: torch.Tensor):
        # print("l2 loss")
        # print("Q_sig:", Q_sig.shape)
        # print()

        Z: torch.Tensor = self.classifier(Q_sig)
        # print("Z", Z.shape)
        Z_significant = []
        for y, z in zip(y_true, Z):
            Z_significant.append(z[y])
        Z_significant = torch.concat(Z_significant)
        # print("Z_significant:", Z_significant.shape)

        y_pred = -torch.log(F.softmax(Z_significant, dim=-1))
        # print("y_pred", y_pred.shape)
        l2 = torch.sum(y_pred)
        # print("l2:", l2.shape)
        return l2


    def forward(self, support: torch.Tensor, query: torch.Tensor, y_true: torch.Tensor):
        # print("support:", support.shape, "query:", query.shape, "y_true:", y_true.shape)
        P_features, Q_features = self.can(support, query)
        # print("P_features:", P_features.shape, "Q_features:", Q_features.shape)
        Q_significant = []
        P_significant = []
        for y, q, p in zip(y_true, Q_features, P_features):
            Q_significant.append(q[y])
            P_significant.append(p[y])
        Q_significant = torch.concatenate(Q_significant, dim=0)
        P_significant = torch.concatenate(P_significant, dim=0)
        # print("P_significant:", P_significant.shape, "Q_significant:", Q_significant.shape)
        l1 = self.L1_loss(P_significant, Q_significant, y_true)
        # print("L1 loss", l1.item())
        l2 = self.L2_loss(Q_significant, y_true)
        # print("L2 loss", l2.item())
        loss = self.lamb*l1 + l2
        return loss

In [17]:
b = 1
M = 6
K = 5
c = 3
height, width = 32, 32
can = CrossAttentionNetwork()
criterion = CANLoss(can, n_classes=M)

support = torch.ones(M, M, K, c, height, width)
query = torch.ones(M, c, height, width)
y_true = torch.randint(0, 5, (M, 1))
loss = criterion(support, query, y_true)
loss.backward()

support: torch.Size([6, 6, 5, 3, 32, 32]) query: torch.Size([6, 3, 32, 32]) y_true: torch.Size([6, 1])
P_features: torch.Size([6, 6, 512, 9]) Q_features: torch.Size([6, 6, 512, 9])
P_significant: torch.Size([6, 512, 9]) Q_significant: torch.Size([6, 512, 9])


In [214]:
a = torch.arange(30).view(3, 5, 2)
idx = torch.tensor([0, 1])
a, a[idx].shape, idx.shape

(tensor([[[ 0,  1],
          [ 2,  3],
          [ 4,  5],
          [ 6,  7],
          [ 8,  9]],
 
         [[10, 11],
          [12, 13],
          [14, 15],
          [16, 17],
          [18, 19]],
 
         [[20, 21],
          [22, 23],
          [24, 25],
          [26, 27],
          [28, 29]]]),
 torch.Size([2, 5, 2]),
 torch.Size([2]))

In [257]:
b = 32
M = 5
K = 5
c = 3
height, width = 32, 32
can = CrossAttentionNetwork().cuda()
criterion = CANLoss(can, n_classes=M).cuda()

support = torch.ones(b, M, K, c, height, width).cuda()
query = torch.ones(b, c, height, width).cuda()
y_true = torch.randint(0, 5, (b, 1)).cuda()
loss = criterion(support, query, y_true)
loss.backward()

Utils

In [224]:
import numpy as np
import torch
import random


def set_seed(seed: int):
    random.seed(seed)
    np.random.RandomState(seed)
    torch.manual_seed(seed)

def get_device():
    return "cuda:0" if torch.cuda.device_count() > 0 else 'cpu'


Data prep

In [225]:
from torch.utils.data import Sampler
import random

class FewShotSampler(Sampler):
    def __init__(self, data_source, n_shots: int, shuffle: bool = True):
        self.data_source = data_source
        self.n_shots = n_shots
        self.classes = self._get_classes()
        self.shuffle = shuffle


    def _get_classes(self):
        classes = dict()
        for i, (_, label) in enumerate(self.data_source):
            if label not in classes:
                classes[label] = []
            classes[label].append(i)

        return classes


    def __iter__(self):
        indices = []
        n = len(self.data_source)
        n_classes = len(self.classes)
        episodes = n // ((self.n_shots + 1) * n_classes)

        for i in range(episodes):
            query = []
            for c in self.classes:
                class_ind = self.classes[c]
                lower = i*(self.n_shots + 1)
                upper = (i+1)*(self.n_shots + 1) - 1
                support_one_class = class_ind[lower : upper]
                query.append(class_ind[upper])
                if self.shuffle:
                    random.shuffle(support_one_class)
                indices.extend(support_one_class)
            if self.shuffle:
                random.shuffle(query)
            indices.extend(query)
        return iter(indices)

    def __len__(self):
        return len(self.data_source)

In [226]:
import os
from torch.utils.data import DataLoader, Subset, SubsetRandomSampler
import torchvision
from torchvision import transforms
import shutil


CINIC_MEAN = [0.47889522, 0.47227842, 0.43047404]
CINIC_STD = [0.24205776, 0.23828046, 0.25874835]


def prepare_data_folder(data_path: str, few_shot_path: str):
    subsets = ["train", "valid", "test"]
    classes = {"train": [ 'bird', 'cat', 'deer', 'dog', 'frog', 'horse'],
               "valid": ['airplane','automobile'],
               "test": [ 'ship','truck']
    }
    [os.makedirs(os.path.join(few_shot_path, sub), exist_ok=True) for sub in subsets]
    [[os.makedirs(os.path.join(few_shot_path, sub, c), exist_ok=True) for c in classes[sub]]
     for sub in subsets]
    for sub in subsets:
        old_sub_path = os.path.join(data_path, sub)
        new_sub_path = os.path.join(few_shot_path, sub)
        for c in classes[sub]:
            old_dir = os.path.join(old_sub_path, c)
            new_dir = os.path.join(new_sub_path, c)
            imgs = os.listdir(old_dir)
            random.shuffle(imgs)
            # 600 support imgs -> 120 batches of 5-way
            # 120 imgs for query
            # 720 in total
            new_imgs = imgs[:720]
            [shutil.copyfile(os.path.join(old_dir, img), os.path.join(new_dir, img))
             for img in new_imgs]



def get_dataset(
        path: str,
        batch_size: int,
        n_shots: int,
        shuffle: bool,
        use_augmentations: bool,
) -> DataLoader:
    augmentations = ([
        transforms.RandomHorizontalFlip(),
        transforms.RandomCrop(32, padding=4),
        transforms.RandomErasing()
    ] if use_augmentations else [])
    transform = transforms.Compose([transforms.ToTensor(),
                                    transforms.Normalize(mean=CINIC_MEAN,std=CINIC_STD),
                                    *augmentations])

    ds = torchvision.datasets.ImageFolder(path, transform=transform)
    sampler = FewShotSampler(ds, n_shots, shuffle=shuffle)
    n_way = len(ds.classes)
    episode_size = batch_size * n_way * (n_shots + 1)
    loader = DataLoader(ds, sampler=sampler, batch_size=episode_size, num_workers=2, pin_memory=True)
    return loader


def get_cinic_few(
        data_path: str,
        batch_size: int,
        n_shots: int,
) -> tuple[DataLoader, DataLoader, DataLoader]:
    train_path = os.path.join(data_path, "train")
    valid_path = os.path.join(data_path, "valid")
    test_path = os.path.join(data_path, "test")

    cinic_train = get_dataset(train_path, batch_size, n_shots, True, True)
    cinic_validation = get_dataset(valid_path, batch_size, n_shots, False, False)
    cinic_test = get_dataset(test_path, batch_size, n_shots, False, False)

    return cinic_train, cinic_validation, cinic_test


In [ ]:
# prepare_data_folder(DATA_PATH, FEW_SHOT_PATH)

In [41]:
n_way = 2
n_shots = 5
ds = torchvision.datasets.ImageFolder(os.path.join(FEW_SHOT_PATH, "test"), transform=transforms.ToTensor())
print(len(ds))
sampler = FewShotSampler(ds, n_shots)
loader = DataLoader(ds, sampler=sampler, batch_size=(n_way * n_shots + n_way))

for samples, labels in loader:
    support = samples[:n_way * n_shots]
    support_labels = labels[:n_way * n_shots]
    support_labels = support_labels.view(1, n_way, n_shots)
    print(support_labels)
    break
    # print(samples.shape, label.shape)


1440
0 : 720
1 : 720
1440
1440 120
0 0 (l, u): 0 5
support: 5
0 1 (l, u): 0 5
support: 5
query; 2
1 0 (l, u): 6 11
support: 5
1 1 (l, u): 6 11
support: 5
query; 2
2 0 (l, u): 12 17
support: 5
2 1 (l, u): 12 17
support: 5
query; 2
3 0 (l, u): 18 23
support: 5
3 1 (l, u): 18 23
support: 5
query; 2
4 0 (l, u): 24 29
support: 5
4 1 (l, u): 24 29
support: 5
query; 2
5 0 (l, u): 30 35
support: 5
5 1 (l, u): 30 35
support: 5
query; 2
6 0 (l, u): 36 41
support: 5
6 1 (l, u): 36 41
support: 5
query; 2
7 0 (l, u): 42 47
support: 5
7 1 (l, u): 42 47
support: 5
query; 2
8 0 (l, u): 48 53
support: 5
8 1 (l, u): 48 53
support: 5
query; 2
9 0 (l, u): 54 59
support: 5
9 1 (l, u): 54 59
support: 5
query; 2
10 0 (l, u): 60 65
support: 5
10 1 (l, u): 60 65
support: 5
query; 2
11 0 (l, u): 66 71
support: 5
11 1 (l, u): 66 71
support: 5
query; 2
12 0 (l, u): 72 77
support: 5
12 1 (l, u): 72 77
support: 5
query; 2
13 0 (l, u): 78 83
support: 5
13 1 (l, u): 78 83
support: 5
query; 2
14 0 (l, u): 84 89
suppor

In [89]:
ds.classes

['ship', 'truck']

In [50]:
set_seed(1)
ds_train, ds_valid, ds_test = get_cinic_few(FEW_SHOT_PATH, 32, 5)

[tensor([[[[-0.0343, -0.0505, -0.0505,  ...,  0.0000,  0.0000,  0.0000],
           [-0.0181, -0.0505, -0.0505,  ...,  0.0000,  0.0000,  0.0000],
           [-0.0019, -0.0181, -0.0343,  ...,  0.0000,  0.0000,  0.0000],
           ...,
           [-0.7472, -0.6824, -0.0505,  ...,  0.0000,  0.0000,  0.0000],
           [-0.6986, -0.6014, -0.6662,  ...,  0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
 
          [[ 0.1246,  0.1081,  0.1081,  ...,  0.0000,  0.0000,  0.0000],
           [ 0.1739,  0.1410,  0.1410,  ...,  0.0000,  0.0000,  0.0000],
           [ 0.2233,  0.1904,  0.1739,  ...,  0.0000,  0.0000,  0.0000],
           ...,
           [-0.7642, -0.6983, -0.0071,  ...,  0.0000,  0.0000,  0.0000],
           [-0.5996, -0.5008, -0.5831,  ...,  0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
 
          [[-0.6785, -0.6937, -0.6937,  ...,  0.0000,  0.0000,  0.0000],
           [-

In [54]:
samples, labels = next(iter(ds_train))
print(labels.tolist())

[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 0, 1, 2, 3, 4, 5, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 0, 1, 2, 3, 4, 5, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 0, 1, 2, 3, 4, 5, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 0, 1, 2, 3, 4, 5, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 0, 1, 2, 3, 4, 5, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 0, 1, 2, 3, 4, 5, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 0, 1, 2, 3, 4, 5, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 0, 1, 2, 3, 4, 5, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 0, 1, 2, 3, 4, 5, 0, 0, 0, 0, 0, 1, 1, 1, 1, 

In [95]:
ds_train.dataset.classes

['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

In [263]:
class SpecialCrossEntropyLoss(nn.Module):
    def __init__(self):
        super(SpecialCrossEntropyLoss, self).__init__()
        self.logsoftmax = nn.LogSoftmax(dim=1)

    def forward(self, inputs, targets):
        # inputs: (b, query_size, n_classes, h, w)
        # inputs: (b, query_size)
        inputs = inputs.view(inputs.size(0)*inputs.size(1), inputs.size(2), -1)
        targets = targets.view(-1)

        log_probs = self.logsoftmax(inputs)
        print(targets)
        targets = torch.zeros(inputs.size(0), inputs.size(1)).scatter_(1, targets.unsqueeze(1).data.cpu(), 1)
        print(targets)
        targets = targets.unsqueeze(-1)
        targets = targets.cuda()
        print("log_probs:", log_probs.shape, "targets:", targets.shape)
        loss = (-targets * log_probs).mean(0).sum()
        print(-targets*log_probs)
        return loss / inputs.size(2)

### Training time
---

In [227]:
n_way = 6
n_shots = 5
c, h, w = 3, 32, 32

set_seed(1)
ds_train, ds_valid, ds_test = get_cinic_few(FEW_SHOT_PATH, 1, n_shots)

In [236]:
def split_support_query(
        episode_samples: torch.Tensor,
        n_way: int,
        n_shots: int
) -> tuple[torch.Tensor, torch.Tensor]:
    episode_samples = episode_samples
    query = episode_samples[n_way * n_shots:].unsqueeze(0)
    support = episode_samples[:n_way * n_shots].unsqueeze(0)
    return support, query


Warm start

In [264]:
from torch.nn import CrossEntropyLoss

n_way = 6
n_shots = 5

can = CrossAttentionNetwork().cuda()
criterion = SpecialCrossEntropyLoss()
optimizer_params = {
    "lr": 0.1,
}
optimizer = SGD(can.parameters(), **optimizer_params, )
epochs = 25

for epoch in range(epochs):
    losses = []
    for episode_samples, episode_labels in tqdm(ds_train):
        optimizer.zero_grad()
        # print(episode_samples.shape)
        # print(episode_labels.shape)
        support, query = split_support_query(episode_samples, n_way, n_shots)
        support: torch.Tensor = support.cuda()
        y_support = episode_labels[:n_way * n_shots].unsqueeze(0)
        y_support_ohe: torch.Tensor = F.one_hot(y_support, n_way).float().cuda()

        y_query = episode_labels[n_way * n_shots:].unsqueeze(0)
        y_query_ohe: torch.Tensor =  F.one_hot(y_query,n_way).float().cuda()
        query: torch.Tensor = query.cuda()

        # print("SUP:", support.shape)
        # print("QUERY:", query.shape)
        # print("Y_SUP:", y_support_ohe.shape)
        # print("Y_QUERY:", y_query_ohe.shape)

        y_pred, cls_score = can(support, query, y_support_ohe, y_query_ohe)
        # print("y_pred", y_pred.shape, "cls_score", cls_score.shape)
        loss1 = criterion(y_pred, y_query.view(-1))
        loss2 = criterion(cls_score, y_query.view(-1))
        # print("l1:", loss1, "l2: ", loss2.item)
        loss = loss1 + 0.5 * loss2
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
        break
    print(f"Epoch {epoch+1}, loss: {torch.tensor(losses).mean()}")
    print("Losses", losses)
    break

  0%|          | 0/120 [00:00<?, ?it/s]

tensor([0, 4, 3, 1, 5, 2])
tensor([[1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 1., 0., 0., 0.]])
log_probs: torch.Size([6, 6, 9]) targets: torch.Size([6, 6, 1])
tensor([[[1.4144, 1.8716, 2.0096, 1.5667, 1.6312, 1.9034, 1.2339, 1.1492,
          1.6190],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000]],

        [[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0

  0%|          | 0/120 [00:07<?, ?it/s]

Epoch 1, loss: 2.8213820457458496
Losses [2.8213820457458496]


In [252]:
from torch.nn import CrossEntropyLoss

can = CrossAttentionNetwork().cuda()
criterion = SpecialCrossEntropyLoss()
optimizer_params = {
    "lr": 0.1,
}
optimizer = SGD(can.parameters(), **optimizer_params, )
epochs = 25

for epoch in range(epochs):
    losses = []
    for episode_samples, episode_labels in tqdm(ds_train):
        optimizer.zero_grad()
        # print(episode_samples.shape)
        # print(episode_labels.shape)
        support, query = split_support_query(episode_samples, n_way, n_shots)
        support: torch.Tensor = support.cuda()
        y_support = episode_labels[:n_way * n_shots].unsqueeze(0)
        y_support_ohe: torch.Tensor = F.one_hot(y_support, n_way).float().cuda()

        y_query = episode_labels[n_way * n_shots:].unsqueeze(0)
        y_query_ohe: torch.Tensor =  F.one_hot(y_query,n_way).float().cuda()
        query: torch.Tensor = query.cuda()

        # print("SUP:", support.shape)
        # print("QUERY:", query.shape)
        # print("Y_SUP:", y_support_ohe.shape)
        # print("Y_QUERY:", y_query_ohe.shape)

        y_pred, cls_score = can(support, query, y_support_ohe, y_query_ohe)
        # print("y_pred", y_pred.shape, "cls_score", cls_score.shape)
        loss1 = criterion(y_pred, y_query.view(-1))
        loss2 = criterion(cls_score, y_query.view(-1))
        # print("l1:", loss1, "l2: ", loss2.item)
        loss = loss1 + 0.5 * loss2
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
    print(f"Epoch {epoch+1}, loss: {torch.tensor(losses).mean()}")
    print("Losses", losses)
        # break

#         loss = criterion(y_pred, y_query_ohe)

# can_warmed_up = CrossAttentionNetwork()
# can_warmed_up.load_state_dict(can.state_dict())
# criterion_warmed_up = CANLoss(can_warmed_up, n_classes=n_way)
# criterion_warmed_up.load_state_dict(criterion.state_dict())

100%|██████████| 120/120 [00:09<00:00, 12.80it/s]


Epoch 1, loss: 4.283754825592041
Losses [2.8294456005096436, 9.136136054992676, 13.230528831481934, 13.052899360656738, 17.615345001220703, 8.604540824890137, 6.4407958984375, 13.702064514160156, 7.341562271118164, 28.1121826171875, 4.872560024261475, 3.0149474143981934, 13.354408264160156, 28.230745315551758, 8.545151710510254, 6.080874443054199, 11.377700805664062, 4.19000244140625, 3.9890012741088867, 4.6881513595581055, 4.505551338195801, 4.5318169593811035, 4.680295944213867, 4.42927360534668, 3.2379324436187744, 3.5702686309814453, 4.196882247924805, 3.2555174827575684, 5.165862083435059, 4.894817352294922, 3.3822593688964844, 3.249025583267212, 2.931387186050415, 3.2217676639556885, 2.6986289024353027, 3.2894160747528076, 3.72867751121521, 2.5520641803741455, 4.32594108581543, 4.045936107635498, 2.9464457035064697, 3.0410008430480957, 2.6001687049865723, 2.3025436401367188, 2.6005003452301025, 4.34052038192749, 2.542372465133667, 3.8469910621643066, 2.810100555419922, 2.60832762

100%|██████████| 120/120 [00:09<00:00, 13.11it/s]


Epoch 2, loss: 2.7312419414520264
Losses [3.060857057571411, 2.69051456451416, 3.01593279838562, 2.680072784423828, 2.6981589794158936, 2.9113516807556152, 2.809696912765503, 2.7537803649902344, 2.794569253921509, 2.5362462997436523, 2.792789936065674, 2.7387959957122803, 2.6019511222839355, 2.6583056449890137, 2.936633825302124, 2.519298553466797, 2.584510326385498, 2.68925142288208, 2.604865789413452, 2.9561057090759277, 2.5127692222595215, 2.562983989715576, 2.6948673725128174, 2.9041690826416016, 2.5802602767944336, 2.4910404682159424, 2.9335343837738037, 3.267958164215088, 2.98996901512146, 2.750950336456299, 2.7400858402252197, 2.7195911407470703, 2.7591662406921387, 2.755347967147827, 2.903384208679199, 2.6517157554626465, 3.0106372833251953, 2.669735908508301, 2.675886631011963, 2.453761100769043, 2.6791367530822754, 2.6913013458251953, 2.790091037750244, 2.547377586364746, 2.6052567958831787, 2.6911168098449707, 2.4100379943847656, 3.142364740371704, 2.624706983566284, 2.71562

100%|██████████| 120/120 [00:09<00:00, 12.75it/s]


...

Epoch 23, loss: 2.3314285278320312
Losses [2.757357358932495, 2.3198869228363037, 2.3301868438720703, 2.105756998062134, 2.1424505710601807, 2.479609966278076, 2.5505518913269043, 2.0671072006225586, 2.059544324874878, 1.826061725616455, 2.0257980823516846, 3.1022558212280273, 2.0048749446868896, 2.6278316974639893, 2.038682460784912, 2.3142154216766357, 2.4011483192443848, 2.1719818115234375, 2.5211877822875977, 2.1791791915893555, 1.9793322086334229, 2.078101873397827, 2.112499713897705, 2.270103931427002, 2.4766812324523926, 2.275151491165161, 2.0060205459594727, 2.5007858276367188, 2.4270355701446533, 2.1952567100524902, 2.640820026397705, 2.702566146850586, 2.4646894931793213, 2.0377578735351562, 2.437744379043579, 2.361577272415161, 2.3505077362060547, 2.023777961730957, 2.03017520904541, 1.878673791885376, 2.092095375061035, 3.316777229309082, 2.374136447906494, 2.255547285079956, 1.9288136959075928, 2.296382188796997, 2.574056625366211, 2.246173858642578, 2.282146692276001, 2.1

100%|██████████| 120/120 [00:10<00:00, 11.89it/s]


Epoch 24, loss: 2.3047127723693848
Losses [2.715297222137451, 2.1832921504974365, 2.230015993118286, 2.1808207035064697, 2.0945348739624023, 2.6949288845062256, 3.098728656768799, 2.1109657287597656, 2.1861839294433594, 2.042839527130127, 1.841050148010254, 2.6770193576812744, 2.0923755168914795, 2.7517402172088623, 2.2770121097564697, 2.014504909515381, 2.2955403327941895, 1.6851797103881836, 2.4439663887023926, 2.030555486679077, 2.0820066928863525, 1.837900161743164, 2.139676094055176, 2.7609212398529053, 2.3092403411865234, 1.9794446229934692, 1.9717025756835938, 1.823206901550293, 2.398892879486084, 2.143543243408203, 2.5644655227661133, 2.5306334495544434, 2.3532931804656982, 2.2206430435180664, 2.2850863933563232, 2.11296010017395, 2.26796817779541, 1.8263893127441406, 1.9930129051208496, 1.6643390655517578, 1.9138896465301514, 2.4252612590789795, 2.314427614212036, 2.2357635498046875, 1.7105040550231934, 2.0828516483306885, 1.9041626453399658, 2.211092472076416, 1.9311240911483

100%|██████████| 120/120 [00:09<00:00, 12.03it/s]

Epoch 25, loss: 2.300511598587036
Losses [2.7883827686309814, 2.1250288486480713, 2.5206480026245117, 2.463980197906494, 1.959113597869873, 2.3444247245788574, 2.328101873397827, 2.2106728553771973, 2.450039863586426, 2.0417299270629883, 1.8133907318115234, 2.6542890071868896, 1.9374346733093262, 2.774535655975342, 1.9397151470184326, 2.0586814880371094, 2.5984549522399902, 2.064669609069824, 2.660753011703491, 2.323810577392578, 1.8745503425598145, 1.8692831993103027, 2.052159309387207, 2.958265542984009, 2.2840216159820557, 1.8214857578277588, 1.7888915538787842, 1.8428361415863037, 2.049453020095825, 2.3722219467163086, 3.485842704772949, 2.8806281089782715, 2.3901381492614746, 2.6370623111724854, 2.2754034996032715, 2.0976076126098633, 1.9731249809265137, 1.8105762004852295, 2.1374480724334717, 1.7571771144866943, 2.4155192375183105, 2.736812114715576, 2.5160789489746094, 2.0285134315490723, 1.9638195037841797, 2.3086719512939453, 1.9733318090438843, 2.0892961025238037, 2.006710529

In [246]:
print(y_pred.shape, cls_score.shape)
print(cls_score[0])

torch.Size([6, 6, 3, 3]) torch.Size([6, 6, 3, 3])
tensor([[[0.6723, 0.6892, 0.6877],
         [0.6556, 0.6629, 0.6731],
         [0.6663, 0.6526, 0.6691]],

        [[0.6274, 0.6571, 0.6813],
         [0.6301, 0.6362, 0.6410],
         [0.6410, 0.6144, 0.6337]],

        [[0.6694, 0.6798, 0.7027],
         [0.6486, 0.6566, 0.6589],
         [0.6495, 0.6457, 0.6690]],

        [[0.5005, 0.5307, 0.5705],
         [0.5281, 0.5810, 0.5521],
         [0.5444, 0.5394, 0.5345]],

        [[0.6406, 0.6689, 0.6833],
         [0.6351, 0.6397, 0.6448],
         [0.6366, 0.6207, 0.6429]],

        [[0.5691, 0.6060, 0.6315],
         [0.5913, 0.6170, 0.6109],
         [0.6117, 0.5997, 0.5970]]], device='cuda:0',
       grad_fn=<SelectBackward0>)


In [45]:
can = CrossAttentionNetwork().cuda()
criterion = CANLoss(can, n_classes=n_way).cuda()
optimizer_params = {
    "lr": 0.001,
}
optimizer = SGD(criterion.parameters(), **optimizer_params, )
epochs = 10

for epoch in range(epochs):
    losses = []
    for episode_samples, episode_labels in tqdm(ds_train):
        optimizer.zero_grad()
        # print(episode_samples.shape)
        # print(episode_labels.shape)
        y_true: torch.Tensor = episode_labels[n_way * n_shots:].unsqueeze(-1).cuda()
        support, query = split_support_query(episode_samples, n_way, n_shots)
        support: torch.Tensor = support.cuda()
        query: torch.Tensor = query.cuda()

        # print(support.shape)
        # print(query.shape)
        # print(y_true.shape)

        loss = criterion(support, query, y_true)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
    print(f"Epoch {epoch+1}, loss: {torch.tensor(losses).mean()}")
    print("Losses", losses)

can_warmed_up = CrossAttentionNetwork()
can_warmed_up.load_state_dict(can.state_dict())
criterion_warmed_up = CANLoss(can_warmed_up, n_classes=n_way)
criterion_warmed_up.load_state_dict(criterion.state_dict())

100%|██████████| 120/120 [00:14<00:00,  8.22it/s]


Epoch 1, loss: 198.50894165039062
Losses [192.42591857910156, 237.9314422607422, 237.76531982421875, 245.19598388671875, 221.25787353515625, 236.29574584960938, 239.2960968017578, 289.90252685546875, 234.09927368164062, 219.93170166015625, 228.55364990234375, 350.619873046875, 185.3072967529297, 324.6642761230469, 256.7618408203125, 210.46917724609375, 268.48870849609375, 251.00247192382812, 301.4913330078125, 205.4044952392578, 210.89247131347656, 223.46502685546875, 184.7008514404297, 223.15823364257812, 219.7668914794922, 250.71609497070312, 214.39227294921875, 212.18272399902344, 245.47213745117188, 215.8321533203125, 192.01109313964844, 196.920166015625, 247.02711486816406, 205.81088256835938, 184.94223022460938, 197.227783203125, 194.1817626953125, 183.76893615722656, 195.86712646484375, 233.53994750976562, 205.96414184570312, 185.79127502441406, 193.08319091796875, 187.62911987304688, 189.32125854492188, 181.9144287109375, 183.29896545410156, 191.9380340576172, 182.1277313232422

100%|██████████| 120/120 [00:15<00:00,  7.99it/s]


Epoch 2, loss: 178.96128845214844
Losses [179.9770050048828, 178.4154052734375, 178.51812744140625, 179.78636169433594, 179.15972900390625, 179.83270263671875, 178.6851806640625, 178.74208068847656, 178.71200561523438, 178.9686279296875, 180.3282012939453, 178.48582458496094, 178.55201721191406, 179.2447509765625, 179.7429962158203, 179.1844940185547, 178.46884155273438, 178.6881103515625, 178.9486083984375, 178.95611572265625, 178.5220184326172, 178.22509765625, 178.27886962890625, 179.30364990234375, 178.673095703125, 181.0144805908203, 179.02105712890625, 178.48541259765625, 178.2066650390625, 178.20907592773438, 180.08395385742188, 184.88461303710938, 178.8894500732422, 178.96788024902344, 198.44859313964844, 178.3725128173828, 181.89874267578125, 178.21633911132812, 178.33876037597656, 178.2165069580078, 181.2938995361328, 178.37440490722656, 179.03564453125, 178.23912048339844, 180.42269897460938, 178.78286743164062, 178.5854949951172, 178.0987548828125, 178.1309051513672, 178.30

100%|██████████| 120/120 [00:14<00:00,  8.07it/s]


...

100%|██████████| 120/120 [00:15<00:00,  7.80it/s]


Epoch 9, loss: 178.0452117919922
Losses [178.01480102539062, 178.04432678222656, 178.02935791015625, 177.99264526367188, 178.07205200195312, 178.05343627929688, 178.0538787841797, 178.07432556152344, 178.04615783691406, 178.07736206054688, 178.05311584472656, 178.04226684570312, 178.03492736816406, 177.99124145507812, 178.01351928710938, 178.03269958496094, 178.03759765625, 178.0446319580078, 178.02395629882812, 178.00473022460938, 178.02102661132812, 178.0048828125, 178.1102294921875, 178.03880310058594, 178.0186004638672, 178.0362548828125, 178.043212890625, 178.022705078125, 178.00222778320312, 178.06289672851562, 178.01412963867188, 178.024169921875, 178.16273498535156, 178.02081298828125, 178.01226806640625, 178.0391387939453, 178.02029418945312, 178.0433807373047, 178.02053833007812, 178.0191192626953, 177.997314453125, 178.04519653320312, 178.0301971435547, 178.0805206298828, 178.04190063476562, 178.02972412109375, 178.08657836914062, 178.00050354003906, 178.0264892578125, 178.0

100%|██████████| 120/120 [00:15<00:00,  7.86it/s]

Epoch 10, loss: 178.05142211914062
Losses [178.04055786132812, 178.05270385742188, 177.99264526367188, 177.99847412109375, 178.34024047851562, 178.0435791015625, 178.0194854736328, 178.1002960205078, 177.99256896972656, 178.07032775878906, 178.13406372070312, 178.11837768554688, 178.02764892578125, 178.10238647460938, 178.00669860839844, 178.041748046875, 178.07769775390625, 178.11422729492188, 178.06370544433594, 178.05690002441406, 178.10110473632812, 178.0052490234375, 178.0106201171875, 178.05084228515625, 178.00030517578125, 178.07171630859375, 178.0263214111328, 178.04588317871094, 178.00881958007812, 178.01597595214844, 178.0190887451172, 178.01168823242188, 178.119873046875, 178.03311157226562, 178.04054260253906, 178.1046142578125, 178.0120849609375, 178.01341247558594, 178.0101776123047, 178.07696533203125, 178.03794860839844, 178.0901336669922, 178.0034637451172, 178.0672607421875, 178.11033630371094, 178.00221252441406, 178.06492614746094, 178.0002899169922, 178.06298828125

<All keys matched successfully>

In [51]:
can.load_state_dict(can_warmed_up.state_dict())
criterion.load_state_dict(criterion_warmed_up.state_dict())

<All keys matched successfully>

In [52]:
optimizer_params = {
    "lr": 0.001,
}
optimizer = SGD(criterion.parameters(), **optimizer_params, )
epochs = 10

for epoch in range(epochs):
    losses = []
    for episode_samples, episode_labels in tqdm(ds_train):
        optimizer.zero_grad()
        # print(episode_samples.shape)
        # print(episode_labels.shape)
        y_true: torch.Tensor = episode_labels[n_way * n_shots:].unsqueeze(-1).cuda()
        support, query = split_support_query(episode_samples, n_way, n_shots)
        support: torch.Tensor = support.cuda()
        query: torch.Tensor = query.cuda()

        # print(support.shape)
        # print(query.shape)
        # print(y_true.shape)

        loss = criterion(support, query, y_true)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
    print(f"Epoch {epoch+1}, loss: {torch.tensor(losses).mean()}")
    print("Losses", losses)

100%|██████████| 120/120 [00:15<00:00,  7.82it/s]


Epoch 1, loss: 178.0390625
Losses [178.0507049560547, 178.0585174560547, 178.0208740234375, 178.004150390625, 178.07373046875, 178.0234832763672, 178.04244995117188, 178.07427978515625, 178.02801513671875, 178.05892944335938, 178.058837890625, 178.01821899414062, 178.01966857910156, 177.99734497070312, 177.99365234375, 178.03753662109375, 178.0151824951172, 178.03021240234375, 177.99514770507812, 178.02157592773438, 178.07846069335938, 177.9918212890625, 178.06390380859375, 178.0248260498047, 178.0069580078125, 178.07432556152344, 178.0235595703125, 178.06692504882812, 178.04476928710938, 178.00929260253906, 178.039794921875, 178.04180908203125, 178.15969848632812, 178.040771484375, 178.07904052734375, 178.13258361816406, 178.0528564453125, 178.0921630859375, 178.01751708984375, 178.02383422851562, 178.05450439453125, 178.04824829101562, 178.04150390625, 178.0391845703125, 178.11485290527344, 178.02041625976562, 178.0462646484375, 177.999755859375, 178.0139617919922, 178.04159545898438

100%|██████████| 120/120 [00:15<00:00,  7.78it/s]


Epoch 2, loss: 178.03428649902344
Losses [177.99905395507812, 178.03048706054688, 177.99755859375, 178.0045928955078, 178.0302734375, 178.01617431640625, 178.0039520263672, 178.032470703125, 178.02511596679688, 178.03909301757812, 178.0143585205078, 178.08926391601562, 178.0113525390625, 177.986572265625, 178.0303955078125, 178.05819702148438, 178.00157165527344, 178.0264434814453, 178.0155792236328, 178.00390625, 178.060791015625, 177.99758911132812, 178.01333618164062, 178.025634765625, 178.03009033203125, 178.04598999023438, 178.01791381835938, 178.00894165039062, 178.00205993652344, 178.0091552734375, 178.01889038085938, 178.0022430419922, 178.09632873535156, 178.000244140625, 178.03701782226562, 178.06353759765625, 178.0583953857422, 178.061767578125, 178.02517700195312, 177.99606323242188, 178.08358764648438, 178.02828979492188, 178.00985717773438, 178.03439331054688, 178.05552673339844, 178.01019287109375, 178.0233917236328, 177.99880981445312, 178.0674591064453, 178.06016540527

100%|██████████| 120/120 [00:15<00:00,  7.86it/s]


...

Epoch 8, loss: 178.03121948242188
Losses [178.01185607910156, 178.04522705078125, 178.0711669921875, 177.9980010986328, 178.08372497558594, 178.0096435546875, 178.04241943359375, 178.0178985595703, 178.01260375976562, 177.99740600585938, 178.06973266601562, 178.12057495117188, 178.0122528076172, 177.9859619140625, 178.014404296875, 178.0413055419922, 178.00221252441406, 178.02523803710938, 178.04388427734375, 177.99166870117188, 178.0423126220703, 177.9960479736328, 177.9939422607422, 178.00816345214844, 177.99468994140625, 178.05691528320312, 178.0072021484375, 178.06423950195312, 178.0068359375, 177.99371337890625, 178.0346221923828, 178.02120971679688, 178.07481384277344, 178.0118408203125, 178.0138397216797, 178.056640625, 178.0389862060547, 178.01336669921875, 178.03872680664062, 178.0150146484375, 178.08921813964844, 178.043212890625, 178.02508544921875, 178.02047729492188, 178.03164672851562, 178.00733947753906, 178.01055908203125, 177.99057006835938, 178.02093505859375, 178.004

100%|██████████| 120/120 [00:15<00:00,  7.72it/s]


Epoch 9, loss: 178.1092071533203
Losses [177.99659729003906, 178.02719116210938, 178.01803588867188, 177.99168395996094, 178.03231811523438, 178.01815795898438, 178.027099609375, 178.04031372070312, 178.01560974121094, 178.01925659179688, 178.0301513671875, 178.03646850585938, 178.00753784179688, 177.99142456054688, 177.99256896972656, 178.0326690673828, 178.00360107421875, 178.04623413085938, 178.01654052734375, 177.9921112060547, 178.03250122070312, 177.99644470214844, 177.99220275878906, 177.99891662597656, 178.00567626953125, 178.03475952148438, 178.0072479248047, 178.01287841796875, 178.00698852539062, 178.0072784423828, 178.02267456054688, 177.9951171875, 178.0298614501953, 178.00181579589844, 178.01760864257812, 178.0435791015625, 178.0468292236328, 177.99229431152344, 178.0223388671875, 177.99932861328125, 178.0791015625, 178.02761840820312, 178.00466918945312, 178.02659606933594, 178.0367431640625, 178.01199340820312, 178.01841735839844, 177.98748779296875, 178.00242614746094,

100%|██████████| 120/120 [00:16<00:00,  7.29it/s]

Epoch 10, loss: 178.02137756347656
Losses [178.00250244140625, 178.05242919921875, 178.00369262695312, 177.98593139648438, 178.05307006835938, 178.003662109375, 177.99461364746094, 178.04580688476562, 177.991943359375, 178.01319885253906, 178.053955078125, 178.08901977539062, 177.9940185546875, 177.99400329589844, 177.99195861816406, 178.00567626953125, 177.99795532226562, 178.02911376953125, 178.0139617919922, 178.01580810546875, 178.02813720703125, 177.99203491210938, 177.99359130859375, 178.00045776367188, 177.9938507080078, 177.9940185546875, 177.9979248046875, 178.0817413330078, 177.99224853515625, 177.99130249023438, 178.00216674804688, 178.00607299804688, 178.096435546875, 178.2023468017578, 178.0006866455078, 178.22238159179688, 178.0074005126953, 178.00064086914062, 178.0033721923828, 178.0355224609375, 178.0518035888672, 178.01351928710938, 177.99185180664062, 178.04815673828125, 178.0233154296875, 177.9921112060547, 178.00303649902344, 177.9933624267578, 177.99871826171875, 